In [1]:
import numpy as np
import h5py
import cv2
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # "0,1,2,3" # "0"
import matplotlib.pyplot as plt
from copy import deepcopy
from core.median_filter import apply_median_filter_gpu_simple
from core.CD_measure import *
from skimage.metrics import *
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import json
%load_ext autoreload
%autoreload 2

In [2]:
# samsung_images = {}
# with h5py.File("../data/val_Samsung_SNU_patches_SET7_divided_by_fnum.hdf5", 'r') as f:
#     f1_images = np.array(f['F1'])
#     f2_images = np.array(f['F2'])
#     f4_images = np.array(f['F4'])
#     f8_images = np.array(f['F8'])
#     f16_images = np.array(f['F16'])
#     f32_images = np.array(f['F32'])
#     f64_images = np.array(f['F64'])
#     for f_num in ['F1', 'F2', 'F4', 'F8', 'F16', 'F32', 'F64']:
#         samsung_images[f_num] = np.array(f[f_num])

In [3]:
# for key,item in samsung_images.items():
#     print(key,item.shape)

In [4]:
pad = 150
noisy_f_num_list = ['F01', 'F02', 'F04', 'F08']
CD_dict ={}
target = 'F16_v2'
with open(f'./result_data/hole_info_{target}.txt', 'r') as f:
    hole_info = json.load(f)
def return_img(img_dict): 
    img_info = {}
    for set_num in img_dict.keys():
        for f_num in img_dict[set_num].keys():
            for idx,img in enumerate(img_dict[set_num][f_num]):
                img_info = {'set_num' : set_num, 'f_num' : f_num, 'idx' : idx}
                yield img_info, img

In [5]:
img_dict = np.load("./intermediate_result/full_img_dict.npy",allow_pickle=True).item()
CD_dict = {}
collect_metric_list = ["avg_min_CD", "avg_max_CD"]

method_list = ["median"]
print_img,save_img = False, False

In [6]:

for img_info, img in return_img(img_dict):
    if img_info['set_num'] != 'SET05'  and img_info['idx'] != 0 or img_info['f_num'] != 'F32':
        continue

    print(img_info, img.shape) # (1,x,y)
    set_num, f_num, idx = img_info["set_num"], img_info["f_num"], img_info["idx"]
    key = f'{img_info["set_num"]}_{img_info["f_num"]}_{img_info["idx"]:02d}'
    img_hole_info = hole_info[set_num]['F08'][idx]
    CD_dict[key] = {}
    for method in method_list:
        target_model = apply_median_filter_gpu_simple
        print(f"===== {method}  CD measure =====")
        CD_info_ours = CD_process(target_model, img, img_info, img_hole_info, pad,print_img,save_img)
        CD_dict = get_metric(CD_dict, CD_info_ours,key, collect_metric_list, method)
    if img_info['set_num'] == 'SET02':
        break
        print(CD_dict)

In [7]:
CD_dict

{}

In [10]:

for img_info, img in return_img(img_dict):
    if img_info['set_num'] != 'SET05' and img_info['idx'] != 0:
        continue
    
    print(img_info, img.shape) # (1,x,y)
    set_num, f_num, idx = img_info["set_num"], img_info["f_num"], img_info["idx"]
    key = f'{img_info["set_num"]}_{img_info["f_num"]}_{img_info["idx"]:02d}'
    img_hole_info = hole_info[set_num]['F08'][idx]
    
    save_hole_img(img,img_info, img_hole_info,pad)
    if img_info['f_num'] == 'F64':
        break

{'set_num': 'SET01', 'f_num': 'F08', 'idx': 0} (1, 1474, 3010)
{'set_num': 'SET01', 'f_num': 'F16', 'idx': 0} (1, 1474, 3010)
{'set_num': 'SET01', 'f_num': 'F32', 'idx': 0} (1, 1474, 3010)
{'set_num': 'SET01', 'f_num': 'F64', 'idx': 0} (1, 1474, 3010)


In [9]:
sys.exit(-1)

SystemExit: -1

/mnt/ssd/hyun/fbi-net/env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
for f_num in ['F1', 'F2', 'F4', 'F8', 'F16', 'F32']:
    avg_psnr = []
    avg_ssim = []
    for noisy, clean in zip(samsung_images[f_num], samsung_images['F64']):
        denoised_img = apply_median_filter(noisy)
        avg_psnr.append(psnr(clean,denoised_img))
        avg_ssim.append(ssim(clean,denoised_img))
    print(f"==== {f_num} denoised result ====")
    print(f"PSNR : {np.mean(avg_psnr):.6f} SSIM : {np.mean(avg_ssim):.6f}")
    print(f"{np.mean(avg_psnr):.6f}\t{np.mean(avg_ssim):.6f}")